In [1]:
import numpy as np
import cv2
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import salientregions as sr

In [2]:
def get_protrusions(img, filled, SE, lam):
    filled, filled_small = fill_image(img, lam)
    fill_big = cv2.bitwise_and(filled, cv2.bitwise_not(cv2.bitwise_and(img, filled_small)))

In [2]:
#Load the image
path_to_image = '../../../TestData/Binary/Binary_holes.png'
holesim = sr.binarize(cv2.imread(path_to_image), vizualize=False)
path_to_image_islands = '../../../TestData/Binary/Binary_islands.png'
islandsim = sr.binarize(cv2.imread(path_to_image_islands), vizualize=False)
path_to_image_example = '../../../TestData/Binary/Binary_all_types_noise.png'
allim = sr.binarize(cv2.imread(path_to_image_example), vizualize=False)
path_to_image_indpred = '../../../TestData/Binary/Binary_indentations_protrusions.png'
indpredim = sr.binarize(cv2.imread(path_to_image_indpred), vizualize=False)
path_to_image_nested = '../../../TestData/Binary/Binary_nested.png'
nestedim = sr.binarize(cv2.imread(path_to_image_nested), vizualize=False)

In [3]:
img = allim
sr.show_image(img)

In [16]:
#Define parameters and get structuring elements
connectivity = 4
area_factor = 0.05
SE_size_factor = 0.15
SE, lam = sr.get_SE(img, SE_size_factor)
print 'lambda is: %f' % lam
print 'SE size is: %f' % SE.shape[0]

lambda is: 105.000000
SE size is: 43.000000


In [6]:
#get holes
filled, holes = sr.get_holes(img, lam, vizualize=False)
sr.show_image(holes)

In [6]:
#get islands
filledinv, islands = sr.get_islands(img, lam, vizualize=False)
#sr.show_image(islands)

In [5]:
filled, filled_small = sr.fill_image(img, vizualize=False)
SE, lam = sr.get_SE(img, 0.1)
min_area = area_factor * img.shape[0] * img.shape[1]
prots = np.zeros(img.shape, dtype='uint8')
retval, labels, stats, centroids = cv2.connectedComponentsWithStats(filled, connectivity=connectivity)
for i in xrange(1, retval) :
    area =  stats[i, cv2.CC_STAT_AREA]
    ccimage = np.array(255*(labels==i), dtype='uint8')
    if area > min_area:
        wth = cv2.morphologyEx(ccimage, cv2.MORPH_TOPHAT, SE)
        prots += wth
        sr.show_image(wth, 'White top hat')
sr.show_image(prots, 'Indentations with noise')

In [6]:
#Remove noise
prots_nonoise = sr.remove_small_elements(prots, lam)

In [12]:
_, _, indents_true, prots_true = \
    sr.read_matfile('../../../TestData/Binary/Binary_all_types_noise_binregions.mat', vizualize=False)
print 'Indents are correct:', sr.image_diff(prots_true, prots_nonoise)

Indents are correct: False


In [11]:
sr.show_image(filled)
sr.show_image( cv2.morphologyEx(filled, cv2.MORPH_OPEN, SE))
sr.show_image( cv2.morphologyEx(filled, cv2.MORPH_TOPHAT, SE))

In [13]:
sr.show_image(prots_nonoise, "my prots")
sr.show_image(prots_true, "true prots")
sr.show_image(prots_nonoise, "my prots")

In [12]:
 ?cv2.morphologyEx

In [20]:
sr.show_image(SE*255)

In [29]:
import scipy.io as sio
se_matfile = sio.loadmat("../../../TestData/Binary/Binary_all_types_noise_SE.mat")
print se_matfile

{'SE_n': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), '__version__': '1.0', '__header__': 'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Tue Mar 15 18:06:08 2016', '__globals__': []}


In [31]:
se_mat = se_matfile['SE_n']
sr.show_image(se_mat*255)
se_mat

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [34]:
print se_mat.shape
print SE.shape

(41, 41)
(43, 43)
